In [1]:
import pandas as pd
import numpy as np
from catnip.fla_redshift import FLA_Redshift
from sqlalchemy import null
from datetime import datetime

from prefect.blocks.system import Secret
from typing import Dict
from concurrent.futures import ThreadPoolExecutor

In [2]:
def get_redshift_credentials() -> Dict:

    cred_dict = {
        "dbname": Secret.load("stellar-redshift-db-name").get(),
        "host": Secret.load("stellar-redshift-host").get(),
        "port": 5439,
        "user": Secret.load("stellar-redshift-user-name").get(),
        "password": Secret.load("stellar-redshift-password").get(),

        "aws_access_key_id": Secret.load("fla-s3-aws-access-key-id-east-1").get(),
        "aws_secret_access_key": Secret.load("fla-s3-aws-secret-access-key-east-1").get(),
        "bucket": Secret.load("fla-s3-bucket-name-east-1").get(),
        "subdirectory": "us-east-1",

        "verbose": False,
    }

    return cred_dict

with ThreadPoolExecutor(1) as pool:
    rs_creds = pool.submit(lambda: get_redshift_credentials()).result()

In [ ]:
# Tables Used:

# - korepss_opportunities
# - cth_v_historical_ticket
# - cth_historical_attendance
# - ct_customer (& all extra tables)
# - flateamshop
# - location_ticket_type 
    # location_ticket_type_agg AS (
    # SELECT
    #     purchaser_ticketing_id,
    #     arena_level_internal
    # FROM
    #     (SELECT
    #          purchaser_ticketing_id,
    #          arena_level_internal,
    #          ROW_NUMBER() OVER (PARTITION BY purchaser_ticketing_id ORDER BY num_tickets DESC,
    #              CASE arena_level_internal
    #                  WHEN 'Premium' THEN 1
    #                  WHEN 'Lower' THEN 2
    #                  WHEN 'Club' THEN 3
    #                  ELSE 4
    #              END) AS rn
    #      FROM
    #          location_ticket_type)
    # WHERE rn = 1